## És igual que el 1

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' 
import tensorflow as tf

2024-05-22 17:00:05.355477: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import gpt2

In [3]:
from utils import load_encoder_hparams_and_params

### load model: 
- ***encoder***
- ***hyperparameters***
- ***parameters***

In [4]:
model_size = '124M'
n_tokens_to_generate = 40
encoder, hparams, params = load_encoder_hparams_and_params(model_size, 'models')

### hyperparameters

In [5]:
hparams

{'n_vocab': 50257, 'n_ctx': 1024, 'n_embd': 768, 'n_head': 12, 'n_layer': 12}

In [6]:
_ = '''
{
  "n_vocab": 50257, # number of tokens in our vocabulary
  "n_ctx": 1024, # maximum possible sequence length of the input
  "n_embd": 768, # embedding dimension (determines the "width" of the network)
  "n_head": 12, # number of attention heads (n_embd must be divisible by n_head)
  "n_layer": 12 # number of layers (determines the "depth" of the network)
}
'''

 ### Main loop

#### input encoding (tokenizer)

In [7]:
prompt = 'what is your favorite musical band?'

In [8]:
input_ids = encoder.encode(prompt)
input_ids

[10919, 318, 534, 4004, 10530, 4097, 30]

#### output generation

In [9]:
def _generate(inputs, params, n_head, n_tokens_to_generate):

    for _ in range(n_tokens_to_generate):
        
        logits = gpt2.gpt2(inputs, **params, n_head=n_head)  # model forward pass
        next_id = gpt2.np.argmax(logits[-1])                 # greedy sampling
        inputs.append(int(next_id))                     # append prediction to input

    return inputs

In [10]:
n_tokens_to_generate = 11
output_ids = _generate(input_ids, params, hparams['n_head'], n_tokens_to_generate)
output_ids[:-n_tokens_to_generate], output_ids[-n_tokens_to_generate:]

([10919, 318, 534, 4004, 10530, 4097, 30],
 [198, 198, 40, 1842, 262, 2647, 286, 262, 44249, 5542, 13])

#### output decoding

In [11]:
print(encoder.decode(output_ids))

what is your favorite musical band?

I love the music of the Grateful Dead.


### Autoregressive generation loop

In [12]:
def _generate_decode(inputs, params, n_head, n_tokens_to_generate):

    print(inputs)
    print(encoder.decode(inputs))
    print()
    
    outputs = []
    for _ in range(n_tokens_to_generate):
        
        logits = gpt2.gpt2(inputs, **params, n_head=n_head)  # model forward pass
        next_id = gpt2.np.argmax(logits[-1])                       # greedy sampling
        
        inputs.append(int(next_id))                           # append prediction to input
        outputs.append(int(next_id))                          # append prediction to output

        # current input/output
        print(inputs)
        print(encoder.decode(outputs))

    return outputs


In [13]:
input_ids = encoder.encode(prompt)
input_ids

[10919, 318, 534, 4004, 10530, 4097, 30]

In [17]:
n_tokens_to_generate = 1
output_ids = _generate_decode(input_ids, params, hparams['n_head'], n_tokens_to_generate)

[10919, 318, 534, 4004, 10530, 4097, 30, 198, 198, 40]
what is your favorite musical band?

I

[10919, 318, 534, 4004, 10530, 4097, 30, 198, 198, 40, 1842]
 love
